In [85]:
%%writefile assignment33.cu
// Задание 3
// Реализуйте CUDA-программу для обработки массива, демонстрирующую коалесцированный и некоалесцированный доступ к глобальной памяти.
// Сравните время выполнения обеих реализаций для массива размером 1 000 000 элементов.

#include <iostream>  // Для стандартного ввода-вывода (cout, endl)
#include <cstdlib>   // Для функции rand()
using namespace std; // Использование стандартного пространства имен

// Определение констант
#define N 1000000        // Размер массива: 1 000 000 элементов
#define BLOCK_SIZE 256   // Количество потоков в одном блоке
#define ITERATIONS 500   // Количество итераций для усреднения времени

// Ядро с коалесцированным доступом к памяти
__global__ void coalesced_kernel(int* input, int* output) {              // __global__ указывает, что функция выполняется на GPU
    int idx = threadIdx.x + blockIdx.x * blockDim.x;                     // Вычисление уникального индекса для каждого потока
    if (idx < N) {                                                       // Проверка выхода за границы массива

        output[idx] = input[idx] * 2 + 1;                                // Коалесцированный доступ: последовательные потоки читают последовательные адреса
    }
}

// Ядро с некоалесцированным доступом к памяти
__global__ void noncoalesced_kernel(int* input, int* output) {           // Ядро GPU для некоалесцированного доступа
    int idx = threadIdx.x + blockIdx.x * blockDim.x;                     // Вычисление уникального индекса для каждого потока
    if (idx < N) {                                                       // Проверка выхода за границы массива

        int stride = 997;                                                // Некоалесцированный доступ: использование большого шага (997 - простое число)

        int bad_idx = (idx * stride) % N;                                // Расчет индекса с нарушением последовательности доступа

        output[idx] = input[bad_idx] * 2 + 1;                            // Выполнение вычислений
    }
}

int main() {                                                             // Основная функция программы
    int *h_input, *h_output1, *h_output2;                                // Объявление указателей на память хоста

    // Выделение pinned memory на хосте для быстрого копирования
    cudaMallocHost(&h_input, N * sizeof(int));                           // Выделение закрепленной памяти для входного массива
    cudaMallocHost(&h_output1, N * sizeof(int));                         // Выделение закрепленной памяти для выходного массива 1
    cudaMallocHost(&h_output2, N * sizeof(int));                         // Выделение закрепленной памяти для выходного массива 2

    // Инициализация входного массива случайными значениями
    for (int i = 0; i < N; i++) {                                        // Цикл по всем элементам массива
        h_input[i] = rand() % 1000;                                      // Инициализация входного массива случайными значениями от 0 до 999
    }

    int *d_input, *d_output1, *d_output2;                                // Объявление указателей на память устройства

    // Выделение памяти на GPU
    cudaMalloc(&d_input, N * sizeof(int));                               // Выделение памяти на GPU для входного массива
    cudaMalloc(&d_output1, N * sizeof(int));                             // Выделение памяти на GPU для выходного массива 1
    cudaMalloc(&d_output2, N * sizeof(int));                             // Выделение памяти на GPU для выходного массива 2

    cudaMemcpy(d_input, h_input, N * sizeof(int), cudaMemcpyHostToDevice); // Копирование данных с CPU на GPU

    int grid_size = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;                     // Расчет количества блоков для запуска

    cudaEvent_t start, stop;                                               // Создание событий для измерения времени
    cudaEventCreate(&start);                                               // Начало замера - создание события start
    cudaEventCreate(&stop);                                                // Конец замера - создание события stop

    float time_coalesced, time_noncoalesced;                               // Переменные для хранения времени выполнения

    // КОАЛЕСЦИРОВАННЫЙ ДОСТУП
    for (int i = 0; i < 5; i++) {                                         // Цикл прогрева GPU (warm-up) - 5 итераций
        coalesced_kernel<<<grid_size, BLOCK_SIZE>>>(d_input, d_output1);  // Запуск ядра с коалесцированным доступом
    }
    cudaDeviceSynchronize();                                              // Ожидание завершения всех операций на GPU

    cudaEventRecord(start);                                               // Начало измерения времени - запись события start

    // Многократный запуск ядра для точного измерения
    for (int i = 0; i < ITERATIONS; i++) {                                // Цикл по ITERATIONS итерациям
        coalesced_kernel<<<grid_size, BLOCK_SIZE>>>(d_input, d_output1);  // Запуск ядра с коалесцированным доступом
    }

    cudaEventRecord(stop);                                                // Конец измерения времени - запись события stop
    cudaDeviceSynchronize();                                              // Ожидание завершения всех операций на GPU

    cudaEventElapsedTime(&time_coalesced, start, stop);                  // Получение времени выполнения в миллисекундах

    // НЕКОАЛЕСЦИРОВАННЫЙ ДОСТУП
    for (int i = 0; i < 5; i++) {                                        // Цикл прогрева GPU (warm-up) - 5 итераций
        noncoalesced_kernel<<<grid_size, BLOCK_SIZE>>>(d_input, d_output2); // Запуск ядра с некоалесцированным доступом
    }
    cudaDeviceSynchronize();                                             // Ожидание завершения всех операций на GPU

    cudaEventRecord(start);                                              // Начало измерения времени - запись события start

    // Многократный запуск ядра с некоалесцированным доступом
    for (int i = 0; i < ITERATIONS; i++) {                               // Цикл по ITERATIONS итерациям
        noncoalesced_kernel<<<grid_size, BLOCK_SIZE>>>(d_input, d_output2); // Запуск ядра с некоалесцированным доступом
    }

    cudaEventRecord(stop);                                               // Конец измерения времени - запись события stop
    cudaDeviceSynchronize();                                             // Ожидание завершения всех операций на GPU

    cudaEventElapsedTime(&time_noncoalesced, start, stop);               // Получение времени выполнения

    // ВЫВОД РЕЗУЛЬТАТОВ

    cout << "Коалесцированный: " << time_coalesced << " мс" << endl;   // Вывод времени коалесцированного доступа
    cout << "Некоалесцированный: " << time_noncoalesced << " мс" << endl; // Вывод времени некоалесцированного доступа

    // Сравнение и вывод разницы
    if (time_coalesced < time_noncoalesced) {                           // Если коалесцированный доступ быстрее
        float ratio = time_noncoalesced / time_coalesced;               // Расчет отношения времени выполнения
        cout << "Коалесцированный быстрее в " << ratio << " раз" << endl; // Вывод результата
    }
    else if (time_noncoalesced < time_coalesced) {                      // Если некоалесцированный доступ быстрее
        float ratio = time_coalesced / time_noncoalesced;               // Расчет отношения времени выполнения
        cout << "Некоалесцированный быстрее в " << ratio << " раз" << endl; // Вывод результата
    }
    else {                                                              // Если время выполнения одинаковое
        cout << "Время выполнения одинаковое" << endl;                  // Вывод сообщения
    }

    // Освобождение pinned memory на хосте
    cudaFreeHost(h_input);                                             // Освобождение закрепленной памяти входного массива
    cudaFreeHost(h_output1);                                           // Освобождение закрепленной памяти выходного массива 1
    cudaFreeHost(h_output2);                                           // Освобождение закрепленной памяти выходного массива 2

    // Освобождение памяти на GPU
    cudaFree(d_input);                                                 // Освобождение памяти GPU входного массива
    cudaFree(d_output1);                                               // Освобождение памяти GPU выходного массива 1
    cudaFree(d_output2);                                               // Освобождение памяти GPU выходного массива 2

    // Уничтожение событий CUDA
    cudaEventDestroy(start);                                           // Уничтожение события start
    cudaEventDestroy(stop);                                            // Уничтожение события stop

    // Завершение программы
    return 0;                                                          // Возврат успешного завершения программы
}

Overwriting assignment33.cu


In [91]:
# Компиляция и запуск в Colab
!nvcc assignment33.cu -o assignment33
!./assignment33

Коалесцированный: 0.092896 мс
Некоалесцированный: 0.102336 мс
Коалесцированный быстрее в 1.10162 раз
